<a href="https://colab.research.google.com/github/piyush1856/company-assignments/blob/main/acadia_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from wordcloud import WordCloud
import os
!pip install fpdf
from fpdf import FPDF

In [ ]:
data_sheet = pd.read_excel("DS_Python_Assignment.xlsx", sheet_name="Data")

In [ ]:
data_sheet.sample(5)

,CUSTOMERID,STATE,LCPCOUNT,PRIVATELABELTENDERFLAG,TENURE_IN_MONTHS,CLOSESTSTOREDISTANCE,FEMALE,AGE,HS_DIPLOMA,SOME_COLLEGE,...,MDAYREV_L3Y,MDAYREV_L4Y,MDAYQTY_L1Y,MDAYQTY_L2Y,MDAYQTY_L3Y,MDAYQTY_L4Y,MDAYSHOPPER_L1Y,MDAYSHOPPER_L2Y,MDAYSHOPPER_L3Y,MDAYSHOPPER_L4Y
8525,13526,IL,2,Y,-2.0,2.706686,0,NaN,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
6571,11572,OR,0,N,12.0,NaN,0,NaN,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
3565,8566,MI,6,N,236.0,3.722687,1,51.0,1,0,...,0.0,0.0,0,0,0,0,0,0,0,0
3749,8750,OK,1,N,9.0,4.043889,0,NaN,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0
1176,6177,NJ,5,N,53.0,5.950750,0,35.0,0,0,...,0.0,0.0,0,0,0,0,0,0,0,0


In [ ]:
data_sheet.shape

(10000, 117)

In [ ]:
# Get the number of unique data types
data_sheet.dtypes.value_counts()

,count
int64,65
float64,49
object,3


## **Function Creation**

In [ ]:
# Function to list down all the columns with missing values and save the result as an image
def missing_values_table(df, filename="missing_values_plot.png"):
    # Calculate missing values per column
    null_values = df.isnull().sum().reset_index(name='null_count')
    null_values.sort_values(by='null_count', ascending=False, inplace=True)
    null_values.reset_index(drop=True, inplace=True)

    # Filter out columns with 0 missing values
    null_values = null_values[null_values['null_count'] > 0]

    # Create a figure with a horizontal bar plot
    plt.figure(figsize=(12, 8))

    # Plot the missing values as a horizontal bar plot
    sns.barplot(x='null_count', y='index', data=null_values, palette="Blues_d")
    plt.title("List of column with missing values")
    plt.xlabel("Number of Missing Values")
    plt.ylabel("Columns")
    plt.xticks(rotation=45)

    # Save the figure as an image
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

    # Return the DataFrame containing missing values information
    return null_values

In [ ]:
# Function to categorize columns by data type and visualize them separately
def categorize_columns_by_dtype(df, numeric_filename="numeric_columns_plot.png", object_filename="object_columns_plot.png"):
    # Categorize columns by data type
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    object_columns = df.select_dtypes(include=['object']).columns

    # Print the categorized columns
    print("Numeric Columns:")
    print(numeric_columns)
    print("\nObject (Categorical) Columns:")
    print(object_columns)

    # Create text for numeric and object columns separately
    numeric_text = " ".join(numeric_columns)
    object_text = " ".join(object_columns)

    # Generate a word cloud for numeric columns with a specific color
    wordcloud_numeric = WordCloud(
        width=800,
        height=400,
        background_color="white",
        colormap="Blues",  # Use 'Blues' colormap for numeric columns
        collocations=False
    ).generate(numeric_text)

    # Save the word cloud for numeric columns as an image
    plt.figure(figsize=(10, 8))
    plt.imshow(wordcloud_numeric, interpolation="bilinear")
    plt.title("Numeric Columns")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(numeric_filename)
    plt.close()

    # Generate a word cloud for object columns with a different color
    wordcloud_object = WordCloud(
        width=800,
        height=400,
        background_color="white",
        colormap="Oranges",  # Use 'Oranges' colormap for object columns
        collocations=False
    ).generate(object_text)

    # Save the word cloud for object columns as an image
    plt.figure(figsize=(10, 8))
    plt.imshow(wordcloud_object, interpolation="bilinear")
    plt.title("Object (Categorical) Columns")
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(object_filename)
    plt.close()


In [ ]:
# Function to remove duplicate columns and visualize the duplicate columns in a word cloud
def remove_duplicate_columns(df, filename="duplicate_columns_plot.png"):
    # List columns with duplicates
    duplicate_columns = df.columns[df.columns.duplicated()].unique()

    # Print DataFrame before removing duplicates
    print("Before removing duplicates:")
    print("Duplicate Columns:", duplicate_columns)
    print("DataFrame shape before removing duplicates:", df.shape)

    # If no duplicate columns are found, create an image showing "No duplicate columns"
    if duplicate_columns.size == 0:
        plt.figure(figsize=(8, 6))
        plt.text(0.5, 0.5, "No duplicate columns", fontsize=20, ha='center', va='center', color='blue')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(filename)
        plt.close()
    else:
        # Generate a word cloud for the duplicate columns based on frequency
        duplicate_text = " ".join(duplicate_columns)

        wordcloud_duplicates = WordCloud(
            width=800,
            height=400,
            background_color="white",
            colormap="Purples",
            collocations=False
        ).generate(duplicate_text)

        # Plot the word cloud for duplicate columns
        plt.figure(figsize=(8, 6))
        plt.imshow(wordcloud_duplicates, interpolation="bilinear")
        plt.title("Duplicate Columns")
        plt.axis("off")

        # Save the word cloud plot as an image
        plt.tight_layout()
        plt.savefig(filename)
        plt.close()

    # Remove duplicate columns by keeping the first occurrence
    df_cleaned = df.loc[:, ~df.columns.duplicated()]

    # Print DataFrame after removing duplicates
    duplicate_columns_after = df_cleaned.columns[df_cleaned.columns.duplicated()].unique()
    print("\nAfter removing duplicates:")
    print("Duplicate Columns:", duplicate_columns_after)
    print("DataFrame shape after removing duplicates:", df_cleaned.shape)

    return df_cleaned


In [ ]:
# Function to remove constant columns and visualize the constant columns in a word cloud
def remove_constant_columns(df, filename="constant_columns_plot.png"):
    # Identify constant columns (columns where all values are the same)
    constant_columns = [col for col in df.columns if df[col].nunique() == 1]

    # Print DataFrame before removing constant columns
    print("Before removing constant columns:")
    print("Constant Columns:", constant_columns)
    print("DataFrame shape before removing constant column:", df.shape)

    # Ensure the directory exists before saving the image
    directory = os.path.dirname(filename)
    if not os.path.exists(directory) and directory != '':
        os.makedirs(directory)

    # Generate a word cloud for the constant columns based on frequency
    constant_text = " ".join(constant_columns) if constant_columns else "No constant columns"

    wordcloud_constant = WordCloud(
        width=800,
        height=400,
        background_color="white",
        colormap="Blues",  # Use 'Blues' colormap for constant columns
        collocations=False
    ).generate(constant_text)

    # Plot the word cloud for constant columns
    plt.figure(figsize=(8, 6))
    plt.imshow(wordcloud_constant, interpolation="bilinear")
    plt.title("Removed Constant Columns")
    plt.axis("off")

    # Save the word cloud plot as an image
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

    # Remove constant columns
    df_cleaned = df.drop(columns=constant_columns)

    # Print DataFrame after removing constant columns
    print("\nAfter removing constant columns:")
    print("DataFrame shape after removing constant column:", df_cleaned.shape)

    return df_cleaned

In [ ]:
# Create box plot to visualise the outliers of all the numeric columns
def create_box_plots(df, num_plots_per_page=12, save_dir="box_plots"):
    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Select numeric columns
    numeric_columns = df.select_dtypes(include=[np.number]).columns

    # Calculate the number of pages based on the number of columns and plots per page
    num_pages = int(np.ceil(len(numeric_columns) / num_plots_per_page))

    for page in range(num_pages):
        start_idx = page * num_plots_per_page
        end_idx = min((page + 1) * num_plots_per_page, len(numeric_columns))
        subset_columns = numeric_columns[start_idx:end_idx]

        # Set up the grid layout for the plots (3 plots per row)
        num_cols = 3
        num_rows = int(np.ceil(len(subset_columns) / num_cols))

        # Set the figure size dynamically based on the number of rows
        plt.figure(figsize=(15, 5 * num_rows))

        for idx, col in enumerate(subset_columns, start=1):
            plt.subplot(num_rows, num_cols, idx)
            plt.boxplot(df[col].dropna(), vert=True, patch_artist=True)
            plt.title(col, fontsize=10)
            plt.xticks([])
            plt.grid(True, linestyle="--", alpha=0.7)

        # Adjust layout to prevent overlap
        plt.tight_layout()

        # Save the plot to a PNG file
        save_path = os.path.join(save_dir, f"box_plot_page_{page + 1}.png")
        plt.savefig(save_path)
        plt.close()


In [ ]:
def remove_outliers_iqr(df):
    # Automatically identify numerical columns
    numerical_columns = df.select_dtypes(include=[np.number]).columns

    for column in numerical_columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1

        # Calculate lower and upper bounds
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Clip the column data to the calculated bounds
        df[column] = np.clip(df[column], lower_bound, upper_bound)

    # Call remove_constant_columns and generate word cloud for constant columns
    df_cleaned = remove_constant_columns(df, filename="visuals_after_clipping_outlier/constant_column/constant_columns_plot.png")

    # Call create_box_plots to generate and save box plots
    create_box_plots(df_cleaned, num_plots_per_page=12, save_dir="visuals_after_clipping_outlier/boxplots")

    # Return the modified DataFrame
    return df_cleaned

In [ ]:
# Create charts for any 6 columns and show their distribution
def create_distribution_charts(df, columns=None, save_dir="visuals_distribution_charts"):
    # If no columns are passed, select 6 random columns
    if columns is None:
        columns = np.random.choice(df.columns, 6, replace=False)

    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Set up the grid layout for plots (2 plots per row)
    num_cols = 2
    num_rows = int(np.ceil(len(columns) / num_cols))

    # Set the figure size dynamically based on the number of rows
    plt.figure(figsize=(15, 5 * num_rows))

    for idx, col in enumerate(columns, start=1):
        plt.subplot(num_rows, num_cols, idx)

        # Check the datatype of the column
        if df[col].dtype in ['int64', 'float64']:
            # Plot histogram for numeric columns
            sns.histplot(df[col].dropna(), kde=True, bins=30, color='skyblue')
            plt.title(f'Histogram of {col}', fontsize=12)
        elif df[col].dtype == 'object':
            # Plot bar plot for categorical columns
            sns.countplot(data=df, x=col, palette='Set2')
            plt.title(f'Bar Plot of {col}', fontsize=12)

        plt.xticks(rotation=45)
        plt.tight_layout()

    # Save the plot to a PNG file
    save_path = os.path.join(save_dir, "distribution_charts.png")
    plt.savefig(save_path)
    plt.close()


## **Generating Report**

In [ ]:
def add_missing_values_section(pdf, df):
    pdf.add_page()
    pdf.set_font("Arial", size=14, style='B')
    pdf.cell(200, 10, txt="List down all the columns with missing values", ln=True, align="C")

    missing_values_filename = "missing_values_plot.png"
    missing_values_df = missing_values_table(df, filename=missing_values_filename)

    if os.path.exists(missing_values_filename):
        pdf.image(missing_values_filename, x=10, y=40, w=190)

In [ ]:
def add_categorize_columns_section(pdf, df):
    pdf.add_page()
    pdf.set_font("Arial", size=14, style='B')
    pdf.cell(200, 10, txt="Categorize columns by their data type", ln=True, align="C")

    numeric_plot_filename = "numeric_columns_plot.png"
    object_plot_filename = "object_columns_plot.png"
    categorize_columns_by_dtype(df, numeric_filename=numeric_plot_filename, object_filename=object_plot_filename)

    if os.path.exists(numeric_plot_filename):
        pdf.image(numeric_plot_filename, x=10, y=40, w=190)
    if os.path.exists(object_plot_filename):
        pdf.image(object_plot_filename, x=10, y=150, w=190)

In [ ]:
def add_remove_duplicate_columns_section(pdf, df):
    pdf.add_page()
    pdf.set_font("Arial", size=14, style='B')
    pdf.cell(200, 10, txt="Remove duplicate columns and visualize", ln=True, align="C")

    duplicate_plot_filename = "duplicate_columns_plot.png"
    df_cleaned = remove_duplicate_columns(df, filename=duplicate_plot_filename)

    if os.path.exists(duplicate_plot_filename):
        pdf.image(duplicate_plot_filename, x=10, y=40, w=190)

    return df_cleaned

In [ ]:
def add_remove_constant_columns_section(pdf, df):
    pdf.add_page()
    pdf.set_font("Arial", size=14, style='B')
    pdf.cell(200, 10, txt="Remove constant columns and visualize", ln=True, align="C")

    constant_plot_filename = "constant_columns_plot.png"
    df_cleaned = remove_constant_columns(df, filename=constant_plot_filename)

    if os.path.exists(constant_plot_filename):
        pdf.image(constant_plot_filename, x=10, y=40, w=190)

    return df_cleaned

In [ ]:
def add_box_plots_section(pdf, df):
    pdf.add_page()
    pdf.set_font("Arial", size=16, style='B')
    pdf.set_y(120)
    pdf.cell(0, 10, txt="Visualize Outliers Using Box Plots", ln=True, align="C")

    box_plots_dir = "box_plots"
    create_box_plots(df, save_dir=box_plots_dir)

    box_plot_files = sorted([f for f in os.listdir(box_plots_dir) if f.endswith('.png')])
    for box_plot_file in box_plot_files:
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        # pdf.cell(200, 10, txt=f"Box Plot - {box_plot_file}", ln=True, align="C")
        pdf.image(os.path.join(box_plots_dir, box_plot_file), x=10, y=40, w=190)


In [ ]:
def add_outlier_treatment_section(pdf, df):
    pdf.add_page()
    pdf.set_font("Arial", size=16, style='B')
    pdf.set_y(120)
    pdf.cell(0, 10, txt="Remove Outliers and Visualize", ln=True, align="C")

    # Remove outliers using IQR
    visuals_dir = "visuals_after_clipping_outlier"
    df_after_outlier_removal = remove_outliers_iqr(df)

    # Visualize constant columns after outlier removal
    constant_plot_after_outliers = os.path.join(visuals_dir, "constant_column", "constant_columns_plot.png")
    if os.path.exists(constant_plot_after_outliers):
        pdf.add_page()
        pdf.set_font("Arial", size=14, style='B')
        pdf.cell(0, 10, txt="Constant Columns After Outlier Removal", ln=True, align="C")
        pdf.image(constant_plot_after_outliers, x=10, y=40, w=190)

    # Visualize box plots after outlier removal
    box_plots_after_outliers_dir = os.path.join(visuals_dir, "boxplots")
    box_plot_files_after_outliers = sorted(
        [f for f in os.listdir(box_plots_after_outliers_dir) if f.endswith('.png')]
    )
    for box_plot_file in box_plot_files_after_outliers:
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        # pdf.cell(200, 10, txt=f"Box Plot After Outlier Removal - {box_plot_file}", ln=True, align="C")
        pdf.image(os.path.join(box_plots_after_outliers_dir, box_plot_file), x=10, y=40, w=190)

    return df_after_outlier_removal

In [ ]:
def add_distribution_charts_section(pdf, df):
    pdf.add_page()
    pdf.set_font("Arial", size=16, style='B')
    pdf.set_y(120)
    pdf.cell(0, 10, txt="Distribution Charts for Selected Columns", ln=True, align="C")

    distribution_charts_dir = "visuals_distribution_charts"
    create_distribution_charts(df, save_dir=distribution_charts_dir)

    distribution_chart_files = sorted(
        [f for f in os.listdir(distribution_charts_dir) if f.endswith('.png')]
    )
    for chart_file in distribution_chart_files:
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        # pdf.cell(200, 10, txt=f"Distribution Chart - {chart_file}", ln=True, align="C")
        pdf.image(os.path.join(distribution_charts_dir, chart_file), x=10, y=40, w=190)

In [ ]:
def generate_report(df, output_pdf="Piyush Tyagi - Output.pdf"):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    add_missing_values_section(pdf, df)
    add_categorize_columns_section(pdf, df)
    df_cleaned = add_remove_duplicate_columns_section(pdf, df)
    df_cleaned = add_remove_constant_columns_section(pdf, df_cleaned)
    add_box_plots_section(pdf, df)
    df_cleaned = add_outlier_treatment_section(pdf, df_cleaned)
    add_distribution_charts_section(pdf, df_cleaned)

    pdf.output(output_pdf)


In [ ]:
generate_report(data_sheet)

Numeric Columns:
Index(['CUSTOMERID', 'LCPCOUNT', 'TENURE_IN_MONTHS', 'CLOSESTSTOREDISTANCE',
       'FEMALE', 'AGE', 'HS_DIPLOMA', 'SOME_COLLEGE', 'BACH_GRAD_DEG',
       'LT_HS_DIPLOMA',
       ...
       'MDAYREV_L3Y', 'MDAYREV_L4Y', 'MDAYQTY_L1Y', 'MDAYQTY_L2Y',
       'MDAYQTY_L3Y', 'MDAYQTY_L4Y', 'MDAYSHOPPER_L1Y', 'MDAYSHOPPER_L2Y',
       'MDAYSHOPPER_L3Y', 'MDAYSHOPPER_L4Y'],
      dtype='object', length=114)

Object (Categorical) Columns:
Index(['STATE', 'PRIVATELABELTENDERFLAG', 'MOSAIC'], dtype='object')
Before removing duplicates:
Duplicate Columns: Index([], dtype='object')
DataFrame shape before removing duplicates: (10000, 117)

After removing duplicates:
Duplicate Columns: Index([], dtype='object')
DataFrame shape after removing duplicates: (10000, 117)
Before removing constant columns:
Constant Columns: ['MARRIED', 'CHILDREN', '7-Cost Only Spend']
DataFrame shape before removing constant column: (10000, 117)

After removing constant columns:
DataFrame shape after remo